### Africa

In [273]:
import pandas as pd
import seaborn as sns
import matplotlib as plt 

In [274]:
#Read train 
deaths = pd.read_csv('data/deaths.csv')
cases = pd.read_csv('data/cases.csv')
deaths.drop(['Province/State'], axis=True)
deaths.shape

(274, 84)

In [275]:
df = pd.read_csv('Global_variable_per_country+Intervention.csv')

### Ebola 

https://www.kaggle.com/imdevskp/ebola-outbreak-20142016-complete-dataset

In [276]:
ebola = pd.read_csv('variable/Africa/ebola_2014_2016_clean.csv')
ebola = ebola[ebola.Date == '2016-03-23']

In [277]:
ebola = ebola[['Country','No. of confirmed deaths', 'No. of confirmed cases']]
ebola.rename(columns={'No. of confirmed deaths':'deaths_ebola',
                        'No. of confirmed cases':'cases_ebola'}, inplace=True)

In [278]:
df = df.merge(ebola, how='left', left_on='Territory', right_on='Country')
df.drop(['country', 'Country'], axis=1, inplace=True)
df["deaths_ebola"] = df["deaths_ebola"].fillna(0)
df["cases_ebola"] = df["cases_ebola"].fillna(0)

## Looking at missing values

In [279]:
df.drop(['Alpha-2 code','Country_x'], axis=1, inplace=True)


In [280]:
df.drop(['WEBNAME'], axis=1, inplace=True)

In [281]:
df["Tests/ million"] = df["Tests/ million"].fillna(0)

In [293]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

In [283]:
df.head(5)

Province/State Country/Region_x    Territory  Population      Lat     Long  \
0            NaN      Afghanistan  Afghanistan  26023100.0  33.0000  65.0000   
1            NaN          Albania      Albania   2895947.0  41.1533  20.1683   
2            NaN          Algeria      Algeria  38700000.0  28.0339   1.6596   
3            NaN          Andorra      Andorra     76965.0  42.5063   1.5218   
4            NaN           Angola       Angola  24383301.0 -11.2027  17.8739   

   1/22/20  1/23/20  1/24/20  1/25/20  ...  \
0        0        0        0        0  ...   
1        0        0        0        0  ...   
2        0        0        0        0  ...   
3        0        0        0        0  ...   
4        0        0        0        0  ...   

   Specialist_surgical_per_1000_2008-18   Tests  Positive  Tests/ million  \
0                                   0.0     0.0       NaN             0.0   
1                                  11.6  2989.0     400.0          1044.0   
2                                  12.1     0.0       NaN             0.0   
3                                  83.1     0.0       NaN             0.0   
4                                   NaN     0.0       NaN             0.0   

   Positive/ thousands  lockdown_date  Type_lockdown  nb_trial  deaths_ebola  \
0                  NaN     24/03/2020           Full       0.0           0.0   
1                 13.4     08/03/2020           Full       0.0           0.0   
2                  NaN     24/03/2020           Full       0.0           0.0   
3                  NaN     16/03/2020           Full       0.0           0.0   
4                  NaN     24/03/2020           Full       0.0           0.0   

   cases_ebola  
0          0.0  
1          0.0  
2          0.0  
3          0.0  
4          0.0  

[5 rows x 152 columns]

### More than 6% missing values

In [284]:
missing_value_df[percent_missing>7]

column_name  \
ROTAC.                                                              ROTAC.   
Hib3.c                                                              Hib3.c   
PAB.cs                                                              PAB.cs   
PCV3.c                                                              PCV3.c   
BCG.cs                                                              BCG.cs   
Physicians_per_1000_2009-18                    Physicians_per_1000_2009-18   
Nurse_midwife_per_1000_2009-18              Nurse_midwife_per_1000_2009-18   
Specialist_surgical_per_1000_2008-18  Specialist_surgical_per_1000_2008-18   
Positive                                                          Positive   
Positive/ thousands                                    Positive/ thousands   

                                      percent_missing  
ROTAC.                                      91.487153  
Hib3.c                                      86.088762  
PAB.cs                                      97.560343  
PCV3.c                                      87.334545  
BCG.cs                                      11.653257  
Physicians_per_1000_2009-18                 60.887620  
Nurse_midwife_per_1000_2009-18              60.887620  
Specialist_surgical_per_1000_2008-18        32.130807  
Positive                                    88.684142  
Positive/ thousands                         88.684142

We will supress all of this variables to predict in a 1st time.

In [285]:
na = list(missing_value_df[percent_missing>7].column_name)

In [286]:
df.drop(na, axis=1, inplace=True)

### Between 1 and  6% missing values

In [287]:
missing_value_df[(percent_missing<7) & (percent_missing>1 )]

column_name  percent_missing
Province/State  Province/State         5.709836
HepB3.                  HepB3.         1.583182
DTP3.c                  DTP3.c         1.271736
MCV2.c                  MCV2.c         1.894628
MCV1.c                  MCV1.c         1.271736
Pol3.c                  Pol3.c         1.271736
nb_beds                nb_beds         2.102258
lockdown_date    lockdown_date         6.436543
Type_lockdown    Type_lockdown         1.842720

In [288]:
# We assume it's 0 if no records related to medecine in a country.
df["nb_beds"] = df["nb_beds"].fillna(0)
df["HepB3."] = df["HepB3."].fillna(0)
df["DTP3.c"] = df["DTP3.c"].fillna(0)
df["MCV2.c"] = df["MCV2.c"].fillna(0)
df["MCV1.c"] = df["MCV1.c"].fillna(0)
df["Pol3.c"] = df["Pol3.c"].fillna(0)

for lockdown_date and Type_lockdown we will put a category others for unknowm. 

In [289]:
df["lockdown_date"] =  df["lockdown_date"].fillna('Unknown')
df["Type_lockdown"] =  df["Type_lockdown"].fillna('Unknown')

### Less than 1% missing values

In [290]:
na =missing_value_df[(percent_missing<1) & (percent_missing!=0 )].column_name

In [291]:
for var in na:
    print(na)
    df.fillna(df[var].mean(), inplace=True)

Jan_Temp                                          Jan_Temp
Feb_temp                                          Feb_temp
Mar_temp                                          Mar_temp
Apr_Temp                                          Apr_Temp
May_temp                                          May_temp
Jun_Temp                                          Jun_Temp
July_Temp                                        July_Temp
Aug_Temp                                          Aug_Temp
Sept_temp                                        Sept_temp
Oct_temp                                          Oct_temp
Nov_Temp                                          Nov_Temp
Dec_temp                                          Dec_temp
Annual_temp                                    Annual_temp
%+65                                                  %+65
15-64pop                                          15-64pop
density                                            density
sex                                                    s

TypeError: can only concatenate str (not "float") to str

In [295]:
df.to_csv('Global_variable_per_country+Intervention+Africa.csv',index=False)